# Verification/Test

In [1]:
# Imports and Settings
import numpy as np
import pickle

from osgeo import gdal, ogr
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

from landsat8 import write_geotiff

from utils import (
    print_cm, test_class_name_to_train_label, train_label_to_test_class_name, 
    extract_test_mask, add_label_from_reference_name,
    separate_files_by_date, CLASS_NAME_TO_INT, INT_TO_CLASS_NAME)


verification_vector_data_path = "real_data/puntos_control/CLASIFICACION_Y_ROI.shp"
verification_pixels_path = 'mask_lote_by_class.pickle'

vote_result_file = 'vote_result_after_using_context.pickle'
output_fname = "verification_with_votation_after_using_context.tiff"

# We need projection and GeoTransform
data_date = "150201"
raster_data_path = "real_data/%s/img/L8_229_82_%s.tif" % (data_date, data_date)
print("Reading the input: %s" % raster_data_path)
try:
    raster_dataset = gdal.Open(raster_data_path, gdal.GA_ReadOnly)
except RuntimeError as e:
    report_and_exit(str(e))
geo_transform = raster_dataset.GetGeoTransform()
proj = raster_dataset.GetProjectionRef()
# ##################################

with open(vote_result_file, 'rb') as fresult:
    votation_result = pickle.load(fresult)

with open(verification_pixels_path, 'rb') as fverification:
     labeled_pixels = pickle.load(fverification)

Reading the input: real_data/150201/img/L8_229_82_150201.tif


In [2]:
# Esto se hace una sola vez
# add_label_from_reference_name(verification_vector_data_path)

In [3]:
rows, cols = votation_result.shape
# test_ds = extract_test_mask(verification_vector_data_path, rows, cols, geo_transform, proj, 'reference')

In [4]:
# test_band = test_ds.GetRasterBand(1)
# labeled_pixels = test_band.ReadAsArray()
# with open(verification_pixels_path, 'wb') as fverification:
#     pickle.dump(labeled_pixels, fverification)

In [5]:
write_geotiff(output_fname, labeled_pixels, geo_transform, proj)
print("Verification image created: %s" % output_fname)

Verification image created: verification_with_votation_after_using_context.tiff


In [6]:
for klass, label in CLASS_NAME_TO_INT.items():
    print("%s [%i]: %i test pixels" % (klass, label, labeled_pixels[labeled_pixels==label].shape[0]))
    print("%s [%i]: %i predicted pixels" % (klass, label, votation_result[votation_result==label].shape[0]))

RASTROJO [6]: 0 test pixels
RASTROJO [6]: 2511 predicted pixels
ALFA [1]: 0 test pixels
ALFA [1]: 0 predicted pixels
MAIZ [2]: 26805 test pixels
MAIZ [2]: 43116 predicted pixels
MONTE [4]: 0 test pixels
MONTE [4]: 0 predicted pixels
PN [5]: 1322 test pixels
PN [5]: 59580996 predicted pixels
MANI [3]: 9122 test pixels
MANI [3]: 3473 predicted pixels
SORGO [8]: 3263 test pixels
SORGO [8]: 787 predicted pixels
SOJA [7]: 94761 test pixels
SOJA [7]: 105333 predicted pixels


In [7]:
for_verification = labeled_pixels.nonzero()
verification_labels = labeled_pixels[for_verification]
predicted_labels = votation_result[for_verification]

In [8]:
cm = metrics.confusion_matrix(verification_labels, predicted_labels)

In [9]:
classes_labels = [train_label_to_test_class_name(str(int(c)))[:3] for c in np.unique(predicted_labels)]
print_cm(cm, classes_labels)


   	MAI	MAN	 PN	RAS	SOJ	SOR	
MAI	10814	382	1081	1482	13046	  0	
MAN	2222	1596	198	  0	5106	  0	
 PN	  0	487	290	384	161	  0	
RAS	  0	  0	  0	  0	  0	  0	
SOJ	13764	874	1277	614	77614	618	
SOR	534	  0	319	  0	2252	158	


In [10]:
print("Classification accuracy: %f" %
      metrics.accuracy_score(verification_labels, predicted_labels))

Classification accuracy: 0.668810


In [11]:
print("Classification report:\n%s" %
      metrics.classification_report(verification_labels, predicted_labels,
                                    target_names=classes_labels))

Classification report:
             precision    recall  f1-score   support

        MAI       0.40      0.40      0.40     26805
        MAN       0.48      0.17      0.26      9122
         PN       0.09      0.22      0.13      1322
        RAS       0.00      0.00      0.00         0
        SOJ       0.79      0.82      0.80     94761
        SOR       0.20      0.05      0.08      3263

avg / total       0.67      0.67      0.66    135273



/home/jbigatti/.virtualenvs/satimg/lib/python3.4/site-packages/sklearn/metrics/classification.py:1076: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
